In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import os 
from PIL import Image

In [2]:
def get_jpg_path(file_dir='datasets\\birds\\train\\'):
    path=[]
    clazz=[]
    for root, dirs, files in os.walk(file_dir): 
        if len(dirs)==0:
            cls=root[-3:]
            cls=[int(cls)]*len(files)
            ph=[root+"\\"+p for p in files]
            path.extend(ph)
            clazz.extend(cls)
    return path,clazz
path,clazz=get_jpg_path('datasets\\birds\\train\\')

In [3]:
def enforce(img,corp=(224,224)):
    rs=np.zeros((5,224,224,3))
    box = [(0, 0, 224, 224),
          (76, 0, 300, 224),
          (76, 76, 300, 300),
          (0, 76, 224, 300),
          (38, 38, 262, 262)]
    idx=0
    for b in box:
        region = img.crop(b)
        npre=np.asarray(region)
        rs[idx]=npre
        idx+=1
    return rs

In [4]:
def onehot_label(clazz):
    index=0
    is_insight=set()
    hot=[0]*90
    onehot=[]
    for i in clazz:
        if i not in is_insight:
            is_insight.add(i)
            hot=[0]*90
            hot[index]=1
            index+=1
        onehot.append(hot)
        onehot.append(hot)
        onehot.append(hot)
        onehot.append(hot)
        onehot.append(hot)
    onehot=np.asarray(onehot)
    return onehot

In [5]:
def del_bad_imgs(path):
    index=0
    for p in path:
        im=Image.open(p)
        npim=np.asarray(im)
        if len(npim.shape)==2:
            os.remove(p)
            print(index,npim.shape,"has been removed")
        index+=1
del_bad_imgs(path)

In [6]:
def reszie_jpg(path,shape=(3830,300,300,3)):
    imgs=[]
    index=0
    for p in path:
        im=Image.open(p)
        im=im.resize(shape[1:3])
        ims=enforce(im)
        for i in ims:
            imgs.append(i)
    return np.asarray(imgs,dtype="float")

In [7]:
imgs=reszie_jpg(path)
imgs.shape

(19120, 224, 224, 3)

In [8]:
onehot=onehot_label(clazz)
onehot.shape

(19120, 90)

In [ ]:
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# minMax = MinMaxScaler()
# imgs = minMax.fit_transform(imgs)
imgs=imgs/255
xtr,xte,ytr,yte=train_test_split(imgs,onehot,test_size=0.1,random_state=1)

In [14]:
model = Sequential()
# BLOCK 1
model.add(Conv2D(filters = 64, kernel_size = (9, 9), activation = 'relu', padding = 'same', name = 'block1_conv1', input_shape = (224, 224, 3)))   
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block2_pool1'))
model.add(Conv2D(filters = 128, kernel_size = (9, 9), activation = 'sigmoid', padding = 'same', name = 'block1_conv2'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block1_pool2'))
 
# BLOCK2
model.add(Conv2D(filters = 256, kernel_size = (9, 9), activation = 'relu', padding = 'same', name = 'block2_conv1'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block2_pool3'))
model.add(Conv2D(filters = 512, kernel_size = (9, 9), activation = 'sigmoid', padding = 'same', name = 'block2_conv2'))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2,2), name = 'block2_pool4'))
 
# BLOCK3
# model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv1'))   
# # model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv3'))
# model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block3_pool'))
 
# # BLOCK4
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv1'))   
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv3'))
# model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block4_pool'))
 
# # BLOCK5
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv1'))   
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv3'))
# model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block5_pool'))
 
model.add(Flatten())
model.add(Dense(1024, activation = 'relu', name = 'fc1'))
# model.add(Dropout(rate=0.2))
model.add(Dense(1024, activation = 'relu', name = 'fc2'))
model.add(Dropout(rate=0.2))
model.add(Dense(90, activation = 'softmax', name = 'prediction'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      15616     
_________________________________________________________________
block2_pool1 (MaxPooling2D)  (None, 56, 56, 64)        0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 56, 56, 128)       663680    
_________________________________________________________________
block1_pool2 (MaxPooling2D)  (None, 14, 14, 128)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 14, 14, 256)       2654464   
_________________________________________________________________
block2_pool3 (MaxPooling2D)  (None, 3, 3, 256)         0         
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 3, 3, 512)         10617344  
__________

In [16]:
model.compile(optimizer=Adam(0.1),
              #loss='mse',
              loss='categorical_crossentropy',
              #loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
model.fit(xtr, ytr, epochs=100)

Epoch 1/100
17208/17208 [==============================] - 75s 4ms/sample - loss: 15.9009 - acc: 0.0122
Epoch 2/100
17208/17208 [==============================] - 73s 4ms/sample - loss: 15.9233 - acc: 0.0121
Epoch 3/100
17208/17208 [==============================] - 73s 4ms/sample - loss: 15.9092 - acc: 0.0130
Epoch 4/100
17208/17208 [==============================] - 73s 4ms/sample - loss: 15.9242 - acc: 0.0120
Epoch 5/100
17208/17208 [==============================] - 73s 4ms/sample - loss: 15.9139 - acc: 0.0127
Epoch 6/100
17208/17208 [==============================] - 73s 4ms/sample - loss: 15.9381 - acc: 0.0112
Epoch 7/100
17208/17208 [==============================] - 73s 4ms/sample - loss: 15.9270 - acc: 0.0119
Epoch 8/100
17208/17208 [==============================] - 73s 4ms/sample - loss: 15.9270 - acc: 0.0119
Epoch 9/100
17208/17208 [==============================] - 73s 4ms/sample - loss: 15.9270 - acc: 0.0119
Epoch 10/100
17208/17208 [==============================] - 73s 

KeyboardInterrupt: 

In [ ]:
model.fit(xtr, ytr, epochs=332)

In [ ]:
rs=model.predict(imgs[0:10])
for i in range(len(rs)):
    print(rs[i],onehot[i])

In [ ]:
Dropout?